In [ ]:
import os
import urllib.request
import json
import openai
import random
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_TOKEN")

In [2]:
import replicate

model2version = {
    "text2image" : "f178fa7a1ae43a9a9af01b833b9d2ecf97b1bcb0acfd2dc5dd04895e042863f1",
    "image2image" : "15a3689ee13b0d2616e98820eca31d4c3abcd36672df6afce5cb6feb1d66087d",
    "image2text" : "a4a8bafd6089e1716b06057c42b19378250d008b80fe87caa5cd36d40c1eda90",
}

text2image = replicate.models.get("stability-ai/stable-diffusion")
text2image_version = text2image.versions.list()[0]

image2image = replicate.models.get("stability-ai/stable-diffusion-img2img")
image2image_version = image2image.versions.list()[0]

image2text = replicate.models.get("pharmapsychotic/clip-interrogator")
image2text_version = image2text.versions.list[0]

version2model = {}

for model, label in [(text2image, 'text2image'), (image2image,'image2image'),(image2text, 'image2text')]:
    version2model.update({ver.id: label for ver in model.versions.list()})


ConnectionError: HTTPSConnectionPool(host='api.replicate.com', port=443): Max retries exceeded with url: /v1/models/stability-ai/stable-diffusion/versions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f57ae90a850>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

## Constants

In [ ]:
IMAGE_SAVE_PATH='/var/www/output.designresearch.works/'
ALLOWED_FILE_SIZES= [128, 256, 384, 448, 512, 576, 640, 704, 768, 832, 896, 960, 1024]
SEED_RANGE=[0,9999999]

### Endpoints: /heartbeat

#### Input:
None

#### Output:
{ \
    "status": string \
}

In [ ]:
# GET /heartbeat
print(json.dumps({'status': 'alive'}))

In [ ]:
# ResponseInfo GET /heartbeat
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

### Endpoints: /text_to_text

#### Input:
{ \
    "prompt": string, \
    "max_tokens": int (default: 100), \
    "temperature": float (defaut: 0.0) \
} 

#### Output:
{ \
    "completion": string \
}

In [ ]:
# POST /text_to_text
req = json.loads(REQUEST)
body = req['body']

completion_object = openai.Completion.create(
  model="text-davinci-003",
  prompt=body['prompt'],
  max_tokens=body['max_tokens'] if 'max_tokens' in body else 100,
  temperature=body['temperature'] if 'temperature' in body else 0
)

print(json.dumps({
    'completion': completion_object.choices[0].text
}))

In [ ]:
# ResponseInfo POST /text_to_text
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

### Endpoints: /text_to_image

#### Input:
{ \
    "prompt": string, \
    "cfg_scale": float, \
    "init_img_url": string (URL of the image, default: None), \
    "prompt_strength": float (default: 1.0), \
    "num_outputs": int (default: 1), \
    "width": int (default: 512), \
    "height": int (default: 512), \
    "num_inference_steps": int (default: 1), \
    "seed": int (default random) \,
    "negative_prompt": str (default "")
} 

#### Output:
{ \
    "image_url": string, \
    "seed": int \
}

In [ ]:
# POST /text_to_image
req = json.loads(REQUEST)
body = req['body']

if "width" not in body or body["width"] not in ALLOWED_FILE_SIZES:
    body["width"] = 512

if "height" not in body or body["height"] not in ALLOWED_FILE_SIZES:
    body["height"] = 512
    
if body["height"] * body["height"] > 1024*768:
    body["height"] = 512
    body["width"] = 512
    
seed = body["seed"] if "seed" in body else random.randint(*SEED_RANGE)

output = text2image_version.predict(
    prompt=body['prompt'],
    guidance_scale=body['cfg_scale'],
    init_image=body['init_img_url'] if 'init_img_url' in body else None,
    prompt_strength=body['prompt_strength'] if 'prompt_strength' in body else 1.0,
    num_outputs=int(body['num_outputs']) if 'num_outputs' in body and body['num_outputs'] > 0 else 1,
    height=body["height"],
    width=body["width"],
    num_inference_steps=body["num_inference_steps"] if "num_inference_steps" in body else 1,
    seed=seed,
    negative_prompt=body["negative_prompt"] if "negative_prompt" in body else ""
)

# save images locally
for url in output:
    urllib.request.urlretrieve(url, IMAGE_SAVE_PATH + "_".join(url.split('/')[-2:]))

print(json.dumps({
    'image_url': output,
    'seed': seed
}))

In [ ]:
# ResponseInfo POST /text_to_image
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

### Endpoints: /image_to_image

#### Input:
{ \
    "image_url": string (URL of the image), \
    "prompt": string, \
    "cfg_scale": float, \
    "prompt_strength": float (default: 1.0), \
    "num_outputs": int (default: 1), \
    "width": int (default: 512), \
    "height": int (default: 512), \
    "num_inference_steps": int (default: 1), \
    "seed": int (default: random) \
} 

#### Output:
{ \
    "image_url": string, \
    "seed": int \
}

In [ ]:
# POST /image_to_image
req = json.loads(REQUEST)
body = req['body']

if "width" not in body or body["width"] not in ALLOWED_FILE_SIZES:
    body["width"] = 512

if "height" not in body or body["height"] not in ALLOWED_FILE_SIZES:
    body["height"] = 512
    
if body["height"] * body["height"] > 1024*768:
    body["height"] = 512
    body["width"] = 512
    
seed = body["seed"] if "seed" in body else random.randint(*SEED_RANGE)

output = image2image_version.predict(
    image=body['image_url'],
    prompt=body['prompt'],
    guidance_scale=body['cfg_scale'],
    prompt_strength=body['prompt_strength'] if 'prompt_strength' in body else 1.0,
    num_outputs=int(body['num_outputs']) if 'num_outputs' in body and body['num_outputs'] > 0 else 1,
    height=body["height"],
    width=body["width"],
    num_inference_steps=body["num_inference_steps"] if "num_inference_steps" in body else 1,
    seed=seed
)

# save images locally
for url in output:
    urllib.request.urlretrieve(url, IMAGE_SAVE_PATH + "_".join(url.split('/')[-2:]))

print(json.dumps({
    'image_url': output
    'seed': seed
}))

In [ ]:
# ResponseInfo POST /image_to_image
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

### Endpoints: /image_to_text

#### Input:
{ \
    "image_url": string (URL of the image), \
} 

#### Output:
{ \
    "image_url": string \
}

In [ ]:
# POST /image_to_text
req = json.loads(REQUEST)
body = req['body']

output = image2text_version.predict(
    image=body['image_url'],
    clip_model_name='ViT-H-14/laion2b_s32b_b79k',
    mode='fast'
)

print(json.dumps({
    'image_url': output
}))

In [ ]:
# ResponseInfo POST /image_to_text
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

### Endpoints: /predictions

#### Input:
None

#### Output:
{ \
  'completed_at': string[time], \
  'created_at': string[time], \
  'error': None, \
  'id': string, \
  'input': dict[string, Any] \
  'metrics': {'predict_time': int }, \
  'output': list[string[url]], \
  'started_at': string[time], \
  'source': string[web|api], \
  'status': string, \
  'urls': { \
    'get': string[url], \
    'cancel': string[url] \
          }, \
  'version': string, \
  'webhook_completed': None, \
  'model': string[text2image|image2image|image2text] \
  }

In [ ]:
# GET /predictions
predictions = replicate.predictions._client._request("GET", f"/v1/predictions").json()["results"]
for pred in predictions:
    pred['model'] = version2model[pred['version']]

print(json.dumps(predictions))
    

In [ ]:
# ResponseInfo GET /predictions
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))